In [1]:
# Load required packages.
import os
import torch
import numpy as np
import pandas as pd

from plyfile import PlyData, PlyElement
from helper_functions import load_object, save_object


In [2]:
#path = 'C:/Users/thoma/Desktop/ZHAW MLS/David prod/ADLProtProject/surface'
path = 'C:/Users/david/pyproj/pyg/adl/surfaces'

In [3]:
#List file on so 
from os import walk
def list_files(mpath):
  f = []
  for (dirpath, dirnames, filenames) in walk(mpath):
      f.extend(filenames)
      break
  return(f)

In [4]:
# I think this would do the trick
files_ply = os.listdir(path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/david/pyproj/pyg/adl/surfaces'

In [ ]:
def read_ply_file(folder_path,file_name,requirement):

    plydata = PlyData.read(folder_path + '/' + file_name)
    
    feature = pd.DataFrame(plydata.elements[0].data)
    vertex_indices = pd.DataFrame(plydata.elements[1].data)

    if requirement == 'feature':
        return feature
    if requirement == 'vertex':
        return vertex_indices
    if requirement == 'plyfile':
        return plydata
    if requirement == 'all' :
        return [feature,vertex_indices]

In [ ]:
def read_ply_folder(folder_path):

    files_ply = list_files(path)
    features_prot = [read_ply_file(folder_path,ply_name,'feature')  for ply_name in files_ply]
    vertex_indices = [read_ply_file(folder_path,ply_name,'vertex')  for ply_name in files_ply]
    
    return features_prot,vertex_indices,files_ply


def complex_separation(files_ply,df):
    
    complexes_names = []
    complexes = []
    uniq_names= []
    for f in files_ply :
        sh = f.find('_')
        val = f[0:sh]
        if val not in uniq_names:
            uniq_names.append(val)
    for name in uniq_names:
        prot = []
        prot_names = []
        max_len = 0

        for i in range(len(files_ply)):
            sh = files_ply[i].find('_')
            val = files_ply[i][0:sh]
            if name == val:
                us = len(files_ply[i])
                len_c = len(files_ply[i][sh:us])
                if len_c > max_len:
                    prot.append(df[i])
                    prot_names.append(files_ply[i])
                    max_len = len_c
                else:
                    prot.insert(0,df[i])
                    prot_names.insert(0,files_ply[i])

        complexes.append(prot)
        complexes_names.append(prot_names)
        
    return  complexes,complexes_names
        
            

feature,vetex,ply_files = read_ply_folder(path)
complexes_feature,complexes_names = complex_separation(ply_files,feature)
print(complexes_names)
print(feature)

[]
[]


In [ ]:
from c_ProteinGraph import ProteinGraph
from torch_geometric.utils import dense_to_sparse

source_dir = path
dest_dir = 'C:/Users/david/pyproj/pyg/adl/surfaces_graphs'

for complex in complexes_names: 

    if len(complex) != 3: # Check if there are three files of the complex
        continue
    else:
        for file in complex[0:2]: # only do it for the subunits, not for the complex
            name = file[0:-4]

            if f'{name}.pkl' not in os.listdir(dest_dir): #Only do it for proteins that are not yet in source dir

                features, edges = read_ply_file(path, file,'all')
                    
                pos = features.loc[:,["x","y","z"]].to_numpy()

                features = features.loc[:,["charge","hbond","hphob", "iface", 'nx', 'ny', 'nz']]

                    
                ## Translate edges into a adjacency matrix and then edge_index
                triangles = np.asarray([list(triangle[0]) for _,triangle in edges.iterrows()])
                n_nodes = pos.shape[0]
                adj = np.identity(n_nodes)
                for n1, n2, n3 in triangles:
                        
                    # edge1
                    adj[n1][n2] = 1
                    adj[n2][n1] = 1 # set location in adj = 1

                    #edge2
                    adj[n1][n3] = 1
                    adj[n3][n1] = 1

                    #edge3
                    adj[n2][n3] = 1
                    adj[n3][n2] = 1

                adj = torch.from_numpy(adj)
                edge_index, _ = dense_to_sparse(adj)


                protein = ProteinGraph(features=features, pos=pos, edge_index=edge_index, name=name)
                save_object(protein, dest_dir + f'/{name}.pkl')

In [4]:
from c_ProteinGraph import ProteinGraph

path = 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces/surfaces_graphs'

files_graph_name = list_files(path)
files_graph = []


for f in files_graph_name:
    files_graph.append(load_object(path + '/' + f))

print(files_graph[45])


 


NameError: name 'list_files' is not defined

In [5]:
from c_ProteinGraph import ProteinGraph

path = 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces/surfaces_graphs'

files_graph_name = list_files(path)
files_graph = []


files_graph.append(load_object(path + '/' + files_graph_name[5]))

print(files_graph[0])


 

            Number of Nodes: 2612
            Features: (2612, 7)
            Edge_Index: (2, 18284)
            Coordinates of Nodes: (2612, 3)
            Protein Name: 1AGQ_D


In [6]:

graph_prot_1 = files_graph[0]


In [7]:
print(graph_prot_1)



            Number of Nodes: 2612
            Features: (2612, 7)
            Edge_Index: (2, 18284)
            Coordinates of Nodes: (2612, 3)
            Protein Name: 1AGQ_D


In [8]:
import f_extract_patches as extract
import numpy as np

In [12]:
num_node = graph_prot_1.features.shape[0]

pacthes = extract.extract_patches(graph_prot_1,20,2)

print(pacthes)



from f_visualize_pointcloud import visualize_pointcloud

#colors_protein = np.zeros_like(graph_prot_1.pos)
#colors_protein[:] = [1,0,0]
#colors_protein[subset] = [0,0,1]



#visualize_pointcloud(graph_prot_1.pos, colors = colors_protein)




[[tensor([ 312,  315,  321,  349,  806,  889,  965, 1173, 1218, 1242, 1265, 1448,
        1658, 1683, 1904, 1951, 2229, 2345, 2355, 2359]), tensor([[ 312,  312,  312,  312,  312,  312,  312,  315,  315,  315,  315,  315,
          315,  321,  321,  321,  321,  321,  349,  349,  349,  349,  349,  349,
          806,  806,  806,  806,  889,  889,  889,  889,  889,  889,  889,  965,
          965,  965,  965,  965, 1173, 1173, 1173, 1173, 1173, 1218, 1218, 1218,
         1218, 1218, 1242, 1242, 1242, 1242, 1242, 1242, 1242, 1265, 1265, 1265,
         1265, 1448, 1448, 1448, 1448, 1448, 1658, 1658, 1658, 1658, 1658, 1658,
         1658, 1683, 1683, 1683, 1683, 1904, 1904, 1904, 1904, 1951, 1951, 1951,
         1951, 1951, 1951, 1951, 1951, 1951, 1951, 2229, 2229, 2229, 2229, 2229,
         2345, 2345, 2345, 2345, 2345, 2355, 2355, 2355, 2359, 2359, 2359, 2359],
        [ 312,  806, 1218, 1904, 1951, 2229, 2359,  315,  321,  965, 1173, 1242,
         1951,  315,  321,  965, 1242, 1683,  349

In [199]:
print(edge_index.shape)

torch.Size([2, 60])


## Test what is in a pickle file now

In [ ]:
dir = path + '/graphs'
test = load_object(dir + '/1A99_C.pkl')
print(test)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces/graphs/1A99_C.pkl'

In [ ]:
test.num_features()

NameError: name 'test' is not defined

In [ ]:
test.num_nodes()

In [ ]:
test.features

In [ ]:
test.pos

In [ ]:
test.edge_index

In [ ]:
test.name

In [ ]:
test.patches = ((edge_index, label, center_index), (edge_index, label ), ())